In [11]:
import requests
from bs4 import BeautifulSoup
import selenium
import pandas
from selenium import webdriver

In [15]:
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
#options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100)

In [16]:
url = "https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089.htm?filter.iso3Language=eng"
driver.get(url)

In [20]:
html_soup = BeautifulSoup(driver.page_source, "html.parser")
container = html_soup.find_all('class', class_='p-0 mb-0 mb-md-std css-w5wad1 gd-ui-module css-1qsvfgq')
print(contianer.prettify())

NameError: name 'contianer' is not defined

In [7]:
for contianer in soup.find('class', class_='p-0 mb-0 mb-md-std css-w5wad1 gd-ui-module css-1qsvfgq'):
    print(review.prettify())

TypeError: 'NoneType' object is not iterable